## Francisco Teixeira Rocha Aragão - 2021031726
## Lorenzo Carneiro Magalhães - 2021031505

### Implementação do problema de Entity Search - Information Retrieval

Abaixo estão as implementações para cada submissão realizada. Vale destacar que os caminhos podem variar, então é importante OU colocar os dados na entrada correta OU alterar os caminhos no código.

O código está repetido entre cada submissão pois a ideia é que cada uma seja independente testando diferentes elementos.


### Primeira submissão

Essa primeira submissão é bem simples e implementa o método bm25 sobre o corpus de documentos.

O indice é gerenciado manualmente, sendo feito o preprocessamento dos documentos, com as informações sobre IDs e tokens sendo salvas internamente para serem utilizadas no ranking.

Eessa etapa é muito custosa pois o índice é salvo em memória, porém conseguimos rodar localmente. Em outras máquinas isso possivelmente pode ser ajustado.

In [ ]:
import csv
import json
import re
import string
from pathlib import Path
from typing import List

from rank_bm25 import BM25Okapi
from tqdm import tqdm
import nltk

from nltk.tokenize import wordpunct_tokenize

# definindo constantes para normalização dos textos
STOPWORDS = set(nltk.corpus.stopwords.words("english"))
PUNCT = set(string.punctuation)
REMOVE_SPACE = re.compile(r"\s+") # regex para substituir múltiplos espaços por um único espaço

def normalize(text: str) -> List[str]:
     """
     • lower-case
     • tokeniza com wordpunct_tokenize
     • remove stop-words / pontuação
     • descarta tokens de 1 caractere
     """
     text = REMOVE_SPACE.sub(" ", text.lower())
     tokens = [
         t for t in wordpunct_tokenize(text)
         if t not in STOPWORDS and t not in PUNCT and len(t) > 1
     ]
     return tokens




Aqui o corpus é percorrido para gerar o indice.

In [ ]:

# definindo constantes e caminhos para arquivos
DATA_DIR     = Path("data/ir-20251-rc")
CORPUS_PATH  = DATA_DIR / "corpus.jsonl"
TEST_PATH    = DATA_DIR / "test_queries.csv"
SUBM_PATH    = Path("submission.csv")
TOP_K        = 100  # máx. de entidades por query como descrito no enunciado


# salvando estruturas de indice pra armazenar informações do corpus
docs_tokens: List[List[str]] = []
entity_ids: List[str]        = []

with CORPUS_PATH.open(encoding="utf-8") as f:
    for line in tqdm(f, desc="corpus.jsonl"):
        doc = json.loads(line)

        # concatena campos relevantes de cada documento no corpus
        combined = " ".join([
            doc.get("title", ""),
            doc.get("text",  ""),
            " ".join(doc.get("keywords", [])),
        ])

        docs_tokens.append(normalize(combined))
        entity_ids.append(doc["id"])

bm25 = BM25Okapi(docs_tokens)
print(f"Num : {len(entity_ids):,} documentos indexados.\n")



Por fim as queries são processadas utilizando o bm25.

In [ ]:

print("ranking BM25")
rows_out: List[List[str]] = []

# pegando os scores de cada query e salvando os resultados
with TEST_PATH.open(encoding="utf-8") as f:
    
    reader = csv.DictReader(f)
    
    for row in tqdm(reader, desc="test_queries.csv"):
        
        qid, query = row["QueryId"], row["Query"]
        
        q_tokens   = normalize(query)

        scores   = bm25.get_scores(q_tokens)
        best_idx = sorted(range(len(scores)),
                          key=scores.__getitem__, reverse=True)[:TOP_K]
        
        rows_out.extend([[qid, entity_ids[i]] for i in best_idx])

print(f"total  de linhas na saída: {len(rows_out):,}\n")


# escrevendo arquivo de saida com os resultados
with SUBM_PATH.open("w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    
    writer.writerow(["QueryId", "EntityId"])
    writer.writerows(rows_out)

print(f"fim")

### Segunda submissão

Essa submissão é mais robusta do que a primeira, pois além do bm25, são utilizadas outras estratégias como expansão de queries além do  modelo CrossEncoder para realizar o reranking.

O gerenciamento do indice é feito pela biblioteca `pyserini`, o que não é controlado manualmente.

Aqui apenas estou organizando os parametros iniciais e formatando o corpus para o formato adequado.

In [ ]:
import os
import csv, json, tqdm, numpy as np
from pathlib import Path
#from pyserini.search import SimpleSearcher # essa versão está deprecada, porém funciona também
from pyserini.search.lucene import LuceneSearcher
from sentence_transformers import CrossEncoder

# caminhos aos arquivos importantes
DATA      = Path("data/ir-20251-rc")
TEST_FILE = DATA / "test_queries.csv"
SUBM_FILE = Path("submission.csv")
INDEX_DIR = Path("index_entities")


# trabalhando com formatação do corpus para Anserini
path_in  = DATA/ "corpus.jsonl"

os.makedirs("corp_anserini", exist_ok=True)
path_out = Path("corp_anserini", "corpus.jsonl")


with path_in.open(encoding="utf-8") as fin, \
     path_out.open("w", encoding="utf-8") as fout:
    for line in tqdm.tqdm(fin, desc="convert"):
        obj = json.loads(line)
        contents = " ".join([
            obj.get("title",""),
            " ".join(obj.get("keywords", [])),
            obj.get("text","")
        ])
        fout.write(json.dumps({"id": obj["id"], "contents": contents}) + "\n")


Com a biblioteca `pyserini`, o corpus é então indexado e salvo em um diretório especifico. A biblioteca cuida de todo o processo, não sendo feito nada manualmente.
Não encontramos API para isso então o comando é feito diretamente via subprocess.


In [ ]:
import subprocess

# essa variavel de ambiente evita que o java use muita memoria
# instalei o java com : sudo apt install openjdk-17-sdk
os.environ['_JAVA_OPTIONS'] = '-Xms4g -Xmx24g'

# só passo os parametros necessarios e os caminhos para os arquivos
# o numero de threads pode ser ajustado
cmd = [
    'python', '-m', 'pyserini.index.lucene',
    '-collection', 'JsonCollection',
    '-generator', 'DefaultLuceneDocumentGenerator',
    '-input', './corp_anserini',
    '-index', 'index_entities',
    '-threads', '4',
    '-storePositions', '-storeDocvectors', '-storeRaw'
]

subprocess.run(cmd, check=True)

Agora rodo o todo o novo pipeline de ranking:

Bm25 + rm3 (ranking inicial) -> CrossEncoder (atua sobre ranking inicial de documentos) -> Ranking final

In [ ]:

# parametros do pipeline de ranking
CAND_K = 1000   # numero de candidatos que recupero inicialmente
FINAL_K= 100    # numero final de entidades
W_CE = 0.7    # peso do cross-encoder na interpolação com o bm25

# agora sim inicio das tarefas de ranqueamento
searcher = LuceneSearcher(str(INDEX_DIR))
searcher.set_bm25(k1=0.92, b=0.36) # parametros do bm25, k1 controla a sensibilidade ao tamanho do documento, b controla a normalização
searcher.set_rm3(fb_terms=10, fb_docs=50, original_query_weight=0.5) # parâmetros do RM3, fb_terms é o número de termos de feedback, fb_docs é o número de documentos de feedback, original_query_weight é o peso da query original

ce = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

# como vou fazer interpolação dos modelos, preciso normalizar os scores dos metodos para poder agregalos
def normalize_scores(d):
    vals = np.array(list(d.values()))
    return {k: (v - vals.min()) / (np.ptp(vals) + 1e-9) for k, v in d.items()}

rows_out = []

with TEST_FILE.open() as f:
    reader = csv.DictReader(f)
    for row in tqdm.tqdm(reader, desc="Queries"):
        qid, query = row["QueryId"], row["Query"]

        # inicialmente uso rm3 + bm25 para achar candidatos relevantes para a query
        hits = searcher.search(query, CAND_K)
        cand_ids  = [h.docid for h in hits]
        bm25_dict = {h.docid: h.score for h in hits}

        # agora que tenho os candidatos, uso o cross-encoder para re-ranquear esses candidatos
        texts = [searcher.doc(did).raw() for did in cand_ids]
        ce_scores = ce.predict([(query, t) for t in texts], batch_size=32)
        ce_dict = dict(zip(cand_ids, ce_scores))

        # jutno os scores com interpolação
        b_norm = normalize_scores(bm25_dict)
        c_norm = normalize_scores(ce_dict)
        final_scores = {d: W_CE*c_norm[d] + (1-W_CE)*b_norm[d] for d in cand_ids}

        top_ids = sorted(final_scores, key=final_scores.get, reverse=True)[:FINAL_K]
        rows_out.extend([[qid, did] for did in top_ids])

# salvo os resultados obtidos 
with SUBM_FILE.open("w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["QueryId", "EntityId"])
    writer.writerows(rows_out)

print("fim")

## Terceira submissão

Organizando imports e definindo função de preprocessamento.

In [ ]:
import os
import csv, json, tqdm, numpy as np
import re
from pathlib import Path
#from pyserini.search import SimpleSearcher
from pyserini.search.lucene import LuceneSearcher
from sentence_transformers import CrossEncoder
import string
import nltk

from nltk.tokenize import wordpunct_tokenize

import subprocess

# definindo constantes para normalização dos textos
STOPWORDS = set(nltk.corpus.stopwords.words("english"))
PUNCT = set(string.punctuation)
REMOVE_SPACE = re.compile(r"\s+") # regex para substituir múltiplos espaços por um único espaço

def normalize(text: str) -> str:
     """
     • lower-case
     • tokeniza com wordpunct_tokenize
     • remove stop-words / pontuação
     • descarta tokens de 1 caractere
     """
     text = REMOVE_SPACE.sub(" ", text.lower())
     tokens = [
         t for t in wordpunct_tokenize(text)
         if t not in STOPWORDS and t not in PUNCT and len(t) > 1
     ]
     return " ".join(tokens)

Aqui apenas estou organizando os parametros iniciais e formatando o corpus para o formato adequado.

In [ ]:

# caminhos aos arquivos importantes
DATA      = Path("data/ir-20251-rc")
TEST_FILE = DATA / "test_queries.csv"
SUBM_FILE = Path("submission.csv")
INDEX_DIR = Path("index_entities")

# trabalhando com formatação do corpus para Anserini
path_in  = DATA/ "corpus.jsonl"

os.makedirs("corp_anserini", exist_ok=True)
path_out = Path("corp_anserini", "corpus.jsonl")

with path_in.open(encoding="utf-8") as fin, \
     path_out.open("w", encoding="utf-8") as fout:
    for line in tqdm.tqdm(fin, desc="convert"):
        obj = json.loads(line)
        contents = " ".join([
            obj.get("title",""),
            " ".join(obj.get("keywords", [])),
            obj.get("text","")
        ])
        fout.write(json.dumps({"id": obj["id"], "contents": normalize(contents)}) + "\n")



Com a biblioteca `pyserini`, o corpus é então indexado e salvo em um diretório especifico. A biblioteca cuida de todo o processo, não sendo feito nada manualmente.
Não encontramos API para isso então o comando é feito diretamente via subprocess.


In [ ]:

# essa variavel de ambiente evita que o java use muita memoria
# instalei o java com : sudo apt install openjdk-17-sdk
os.environ['_JAVA_OPTIONS'] = '-Xms4g -Xmx24g'

# só passo os parametros necessarios e os caminhos para os arquivos
# o numero de threads pode ser ajustado
cmd = [
    'python', '-m', 'pyserini.index.lucene',
    '-collection', 'JsonCollection',
    '-generator', 'DefaultLuceneDocumentGenerator',
    '-input', './corp_anserini',
    '-index', 'index_entities',
    '-threads', '4', 
    '-storePositions', '-storeDocvectors', '-storeRaw'
]

subprocess.run(cmd, check=True)

A ideia agora foi alterar alguns parametros, como o numero de documentos retornados pelo bm25 (que será usado no cross encoder). Também foi alterado o peso da interpolação e os parametros do Bm25.

In [ ]:
# parametros do pipeline de ranking
CAND_K = 500   # numero de candidatos que recupero inicialmente
FINAL_K= 100    # numero final de entidades
W_CE = 0.8    # peso do cross-encoder na interpolação com o bm25

# agora sim inicio das tarefas de ranqueamento
searcher = LuceneSearcher(str(INDEX_DIR))
searcher.set_bm25(k1=1.2, b=0.75) # parametros do bm25, k1 controla a sensibilidade ao tamanho do documento, b controla a normalização
searcher.set_rm3(fb_terms=10, fb_docs=50, original_query_weight=0.6) # parâmetros do RM3, fb_terms é o número de termos de feedback, fb_docs é o número de documentos de feedback, original_query_weight é o peso da query original

ce = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

# como vou fazer interpolação dos modelos, preciso normalizar os scores dos metodos para poder agregalos
def normalize_scores(d):
    vals = np.array(list(d.values()))
    return {k: (v - vals.min()) / (np.ptp(vals) + 1e-9) for k, v in d.items()}

rows_out = []

with TEST_FILE.open() as f:
    reader = csv.DictReader(f)
    for row in tqdm.tqdm(reader, desc="Queries"):
        qid, query = row["QueryId"], row["Query"]

        # inicialmente uso rm3 + bm25 para achar candidatos relevantes para a query
        hits = searcher.search(query, CAND_K)
        cand_ids  = [h.docid for h in hits]
        bm25_dict = {h.docid: h.score for h in hits}

        # agora que tenho os candidatos, uso o cross-encoder para re-ranquear esses candidatos
        texts = [searcher.doc(did).raw() for did in cand_ids]
        ce_scores = ce.predict([(query, t) for t in texts], batch_size=32)
        ce_dict = dict(zip(cand_ids, ce_scores))

        # jutno os scores com interpolação
        b_norm = normalize_scores(bm25_dict)
        c_norm = normalize_scores(ce_dict)
        final_scores = {d: W_CE*c_norm[d] + (1-W_CE)*b_norm[d] for d in cand_ids}

        top_ids = sorted(final_scores, key=final_scores.get, reverse=True)[:FINAL_K]
        rows_out.extend([[qid, did] for did in top_ids])

# salvo os resultados obtidos 
with SUBM_FILE.open("w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["QueryId", "EntityId"])
    writer.writerows(rows_out)

print("fim")

### Quarta submissão

Como essa submissão apenas altera os scores finais, não há a necessidade de trabalhar com o corpus novamente igual nas últimas etapas, então essa parte foi ignorada.

Assim, agora essa submissão apenas usa o score final totalmente dependente do modelo de cross encoder

In [ ]:
# parametros do pipeline de ranking
CAND_K = 1000   # numero de candidatos que recupero inicialmente
FINAL_K= 100    # numero final de entidades

# agora sim inicio das tarefas de ranqueamento
searcher = LuceneSearcher(str(INDEX_DIR))
searcher.set_bm25(k1=1.2, b=0.75) # parametros do bm25, k1 controla a sensibilidade ao tamanho do documento, b controla a normalização
searcher.set_rm3(fb_terms=12, fb_docs=50, original_query_weight=0.6) # parâmetros do RM3, fb_terms é o número de termos de feedback, fb_docs é o número de documentos de feedback, original_query_weight é o peso da query original

ce = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')


rows_out = []

with TEST_FILE.open() as f:
    reader = csv.DictReader(f)
    for row in tqdm.tqdm(reader, desc="Queries"):
        qid, query = row["QueryId"], row["Query"]

        # inicialmente uso rm3 + bm25 para achar candidatos relevantes para a query
        hits = searcher.search(query, CAND_K)
        cand_ids  = [h.docid for h in hits]

        # agora que tenho os candidatos, uso o cross-encoder para re-ranquear esses candidatos
        texts = [searcher.doc(did).raw() for did in cand_ids]
        ce_scores = ce.predict([(query, t) for t in texts], batch_size=32)
        ce_dict = dict(zip(cand_ids, ce_scores))

        # agora só uso os scores do cross-encoder

        final_scores = {d: ce_dict[d] for d in cand_ids}

        top_ids = sorted(final_scores, key=final_scores.get, reverse=True)[:FINAL_K]
        rows_out.extend([[qid, did] for did in top_ids])

# salvo os resultados obtidos 
with SUBM_FILE.open("w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["QueryId", "EntityId"])
    writer.writerows(rows_out)

print("fim")